In [3]:
import json
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models, similarities
# spacy for lemmatization
import spacy
! python -m spacy download en

# Removing Stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.test.utils import datapath


    Linking successful
    /home/ubuntu/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
    /home/ubuntu/anaconda3/lib/python3.7/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
temp_file = datapath("lda_1_percent")
model =  models.LdaModel.load(temp_file)

In [8]:
model